In [ ]:
from opxrd import OpXRD

opxrd = OpXRD.load(root_dirpath='/tmp/opxrd')
opxrd.plot_quantity(attr='spacegroup')

In [ ]:
pattern = opxrd.patterns[0]
pattern.plot(title='Example pattern', save_fpath=)